# Big Data Analytics
Praktikum Sommersemester 2023. <small>Version 1.2</small>

**Aufgabe 2: Abfragen mit Apache Drill** 

Machen Sie sich mit Apache Drill vertraut. Lösen Sie die Aufgaben in `sqlline` oder in einem Jupyter Notebook. Nutzen Sie die markierten Zellen im vorliegenden Notebook `BDA1_A2_Drill` für die Lösung und laden Sie es in Ilias hoch.

----

## Vorbereitung
* Erzeugen Sie den Ordner `work/drill-driver` im JupyterLab Workspace
* Legen Sie eine neue Datei `odbc.ini` dort an. Sie finden ein Beispiel unter `Big_Data_Analytics_1/public/drill-driver/`


In Drill sind mehrere Datenmengen konfiguriert und für Sie verwendbar:

* **dfs.data.\`co2data.tsv\`**<br>DFS Datenquelle in Form eines Datensatz an Sensordaten
* **bdapi.labels**<br>HTTP Datenquelle mit Sensornamen und -Positionen
* **bdapi.sensors**<br>HTTP Datenquelle mit einer fixierten Liste von JSON Objekten mit Sensordaten.
* **bdapi.sensorslastday**<br>HTTP Datenqelle mit einem JSON Objekt, das dynamische Sensordaten anbietet.
* **weather.sunrise**<br>HTTP Storage einer API, die Zeiten des Sonnenauf- und Sonnenuntergangs an einer gegebenen Position (fields lat und lon) zu einem Datum (field date) zurückgibt. Mehr Infos dazu auf der Webseite zur [sunrise-sunset.org/api](https://sunrise-sunset.org/api).

## Aufgabe 2 a

Verschaffen Sie sich einen Überblick über die Sensordaten, die Sie im Drill unter ``dfs.data.`co2data.tsv` `` finden. Beantworten Sie die folgenden
Fragen, indem Sie jeweils eine SQL-Query gegen das Drill Cluster ausführen:
    
1. Wieviele verschiedene Sensoren (angegeben im Feld _source_) enhält die Datenmenge?
2. Wieviele Datenpunkte je Sensor liegen vor? Geben Sie sowohl Sensor als auch Anzahl aus.
3. Bereiten Sie in einer SQL-Query die Werte in einzelnen Spalten so vor, dass sie sinnvolle Datentypen aufweisen:
    1. Sowohl _humidity_, _temperature_ als auch _co2_ sollen als auf zwei Nachkommastellen gerundete Fließkommazahlen verfügbar sein.
    2. Die erste Spalte gibt den Zeitstempel als Unix Epoch mit Mikrosekunden an. Machen Sie daraus einen Drill Timestamp
4. Was ist der höchste, und was der niedrigste Temperaturwert? Beide Werte sollen zusammen in einer Query angefragt werden. Kennzeichnen Sie die beiden Felder mit einem entsprechenden Namen.
5. Was ist der durchschnittliche CO<sub>2</sub>-Wert je Sensor in der Datenmenge?


In [52]:
import pyodbc

def connect(dsn='drill'):
    """opens the connection to given DSN"""
    conn = pyodbc.connect("DSN="+dsn, autocommit=True)
    if not conn.closed:
        print(f"connected to drillbit {dsn}")
        return conn
    else:
        print(f"could not connect to {dsn}")
        return None

con = connect()
query = '''SELECT version FROM sys.version'''
csr = con.cursor()
csr.execute(query)
res = csr.fetchone() # fetchmany fetchall
print(res[0])

connected to drillbit drill
1.20.0


## 2a: 1.) Zählen der Anzahl der Sensoren

- Die folgende SQL-Abfrage zählt die Anzahl der Sensoren in den CO2-Daten, die in der Tabelle co2data.tsv gespeichert sind und gibt diese Anzahl zurück.
- Die Identifikationsnummern(serial_number) der Sensoren sind in der zweiten Spalte (Spaltenindex 1, da die Zählung bei 0 beginnt) der Tabelle gespeichert.
- Da ein Sensor mehrere Messungen durchführen kann, können die Nummern der Sensoren in der Tabelle mehrfach erscheinen. Daher wird das DISTINCT-Schlüsselwort in der COUNT-Funktion verwendet, um nur die einzigartigen Sensoren zu zählen.

In [18]:
# Aufgabe 2a: 1.
query = '''
SELECT COUNT(DISTINCT columns[1]) AS sensoren FROM dfs.data.`co2data.tsv`
'''
csr = con.cursor()
csr.execute(query)
res = csr.fetchall()
for row in res:
    print(row)

(23, )


## 2a: 2.) Zählen der Datenpunkte pro Sensor

- Die folgende Abfrage zählt die Anzahl der Messwerte für jeden Sensor bzw. die Anzahl des Auftretens jedes Sensors.
- Die Abfrage gibt eine Tabelle zurück, in der jede Zeile einen Sensor und die Anzahl der von diesem Sensor aufgezeichneten Datenpunkte darstellt.
- Mit dem Befehl GROUP BY werden die Daten nach Sensornummer geordnet. Für jeden einzelnen Sensor wird dann die Gesamtanzahl seiner Messungen mit COUNT(*) berechnet.

In [21]:
# Aufgabe 2a: 2. 
query = '''
SELECT columns[1] AS serial_number, COUNT(*) AS data_points
FROM dfs.data.`co2data.tsv`
GROUP BY columns[1]
'''
csr = con.cursor()
csr.execute(query)
res = csr.fetchall()
for row in res:
    print(row)

('s_3c6105d3abae_', 2)
('s_e8db84c5f33d', 83)
('s_8caab57a6dd9_', 6)
('s_8caab57a6dd9_288065', 2781611)
('s_d8bfc0147061_', 1)
('s_10521c01cf19_262520', 385103)
('s_8caab57c3e19_282028', 1561045)
('s_10521c0202ab_284839', 2064)
('s_8caab57a6dd9', 11)
('s_e8db84c5f771_', 2)
('s_3c6105d3abae_299589', 1533917)
('s_d8bfc0147061_283903', 578456)
('s_8caab57cc961_', 7)
('s_8caab57c3e19_', 1)
('s_e8db84c5f33d_281913', 2269085)
('s_d8bfc014724e_262793', 2103522)
('s_8caab57cc961_284337', 1131861)
('s_8caab57a6dd9_0xdeadbeef', 49)
('s_e8db84c5f771_300390', 1665528)
('s_e8db84c5f33d_', 13)
('serial_number', 1)
('s_10521c01cf19_', 2)
('s_e8db84c5f33d_0xdeadbeef', 1515)


## 2a: 3.) Umwandlung von CO2-Werten, Temperatur und relativer Luftfeuchtigkeit

- Die folgende Abfrage konvertiert die Werte mit "cast" und "decimal" von CO2 , Temperatur  und relative Luftfeuchtigkeit zu Dezimalzahlen.
- Die Abfrage gibt eine Tabelle zurück, in der jede Zeile einen Datensatz darstellt und die Werte für CO2, Temperatur und relative Luftfeuchtigkeit als Dezimalzahlen        dargestellt sind
- Timestamp code: Mit To_Timestamp(TO_TIMESTAMP(CAST(columns[6] AS INT)/1000000) AS timestamp) sollte der Drill timestamp erstellt werden und vorher sollten die Mikrosekunden in Sekunden umgerechnet werden mit dem Faktor 1 Million um für die Timestamp funktion verwendet werden zu können. Danach:AND columns[6] <> 'timestamp'.

In [51]:
# Aufgabe 2a: 3.
query = '''
SELECT 
    ROUND(CAST(columns[3] AS DECIMAL(6, 2)), 2) AS co2_ppm,
    ROUND(CAST(columns[4] AS DECIMAL(6, 2)), 2) AS temperature_celsius,
    ROUND(CAST(columns[5] AS DECIMAL(6, 2)), 2) AS relative_humidity_percent
    
FROM dfs.data.`co2data.tsv`
WHERE columns[3] <> 'co2_ppm'
AND columns[4] <> 'temperature_celsius' 
AND columns[5] <> 'relative_humidity_percent'
LIMIT 10
'''

csr = con.cursor()
csr.execute(query)
res = csr.fetchall()
for row in res:
    print(row)

(Decimal('420.00'), Decimal('23.00'), Decimal('36.00'))
(Decimal('421.00'), Decimal('24.00'), Decimal('32.00'))
(Decimal('420.00'), Decimal('24.00'), Decimal('32.00'))
(Decimal('651.00'), Decimal('20.00'), Decimal('44.00'))
(Decimal('422.00'), Decimal('23.00'), Decimal('36.00'))
(Decimal('422.00'), Decimal('23.00'), Decimal('36.00'))
(Decimal('423.00'), Decimal('23.00'), Decimal('36.00'))
(Decimal('436.00'), Decimal('22.00'), Decimal('41.00'))
(Decimal('653.00'), Decimal('20.00'), Decimal('44.00'))
(Decimal('653.00'), Decimal('20.00'), Decimal('44.00'))


## 2a: 4.) Bestimmung der minimalen und maximalen Temperatur

- Die Funktionen MIN und MAX werden verwendet, um den kleinsten bzw. größten Wert in der Temperaturspalte zu finden.
- Um sicherzustellen, dass die Funktionen MIN und MAX auf gültige Daten angewendet werden, werden Zeilen mit dem Wert 'temperature_celsius' oder 'null' in der Temperaturspalte ausgeschlossen. Das erfolgt durch die WHERE Anweisung.
- min_temperature und max_temperature sind die entsprechenden Kennzeichnungen der Ergebnisfelder

In [5]:
# Aufgabe 2a: 4.
query = '''
SELECT 
    MIN(CAST(columns[4] AS DOUBLE)) AS min_temperature, 
    MAX(CAST(columns[4] AS DOUBLE)) AS max_temperature 
FROM dfs.data.`co2data.tsv`
WHERE columns[4] <> 'temperature_celsius'
AND columns[4] <> 'null'
'''
csr = con.cursor()
csr.execute(query)
res = csr.fetchall()
for row in res:
    print(row)

(-1.0, 36.0)


## 2a: 5.) Durchschnittlicher CO2-Wert pro Sensor

- In dieser Abfrage wird der Durchschnitt vom CO2-ppm Wert für jeden Sensor berechnet. 
- Die AVG-Funktion wird verwendet, um den Durchschnitt der CO2-Werte für jede einzigartige serial_number zu berechnen.
- Auch hier werden ungültige Daten ignoriert, indem die folgenden Zeilen ausgeschlossen werden, die 'co2_ppm', 'null' oder 'serial_number' in den relevanten Spalten haben.
- Schließlich werden die Daten nach der Seriennummer des Sensors gruppiert und der Durchschnitt berechnet.

In [6]:
# Aufgabe 2a: 5.
query = '''
SELECT 
    columns[1] AS sensor_serial_number, 
    AVG(CAST(columns[3] AS DECIMAL(10, 2))) AS avg_co2_ppm
FROM dfs.data.`co2data.tsv`
WHERE columns[3] <> 'co2_ppm'
AND columns[3] <> 'null'
AND columns[1] <> 'serial_number'
GROUP BY columns[1]
'''
csr = con.cursor()
csr.execute(query)
res = csr.fetchall()
for row in res:
    print(row)

('s_8caab57a6dd9_288065', Decimal('654.238816'))
('s_3c6105d3abae_299589', Decimal('833.976671'))
('s_e8db84c5f33d_0xdeadbeef', Decimal('-1.000000'))
('s_8caab57cc961_284337', Decimal('470.793016'))
('s_8caab57a6dd9_', Decimal('983.666667'))
('s_10521c0202ab_284839', Decimal('436.763566'))
('s_e8db84c5f33d', Decimal('1403.228916'))
('s_d8bfc014724e_262793', Decimal('770.455495'))
('s_8caab57a6dd9_0xdeadbeef', Decimal('-1.000000'))
('s_10521c01cf19_', Decimal('467.500000'))
('s_e8db84c5f771_300390', Decimal('954.087763'))
('s_8caab57c3e19_282028', Decimal('876.931584'))
('s_e8db84c5f33d_281913', Decimal('531.210462'))
('s_d8bfc0147061_283903', Decimal('628.633917'))
('s_e8db84c5f33d_', Decimal('488.769231'))
('s_10521c01cf19_262520', Decimal('500.677816'))
('s_e8db84c5f771_', Decimal('863.000000'))
('s_d8bfc0147061_', Decimal('722.000000'))
('s_8caab57c3e19_', Decimal('620.000000'))
('s_8caab57cc961_', Decimal('429.285714'))
('s_3c6105d3abae_', Decimal('832.000000'))


## Aufgabe 2 b

1. Verknüpfen Sie die Daten aus `dfs.data.`co2data.tsv` mit den Daten aus `bdapi.labels`. Geben Sie, wenn möglich, den Sensornamen ( _name_ ) und Besitzer ( _owner_ ) zu jeder Seriennummer aus. Falls Daten fehlen, geben Sie die Seriennummer dennoch aus.<br>Hinweis: Die Seriennummern liegen bei den beiden Datenmengen in unterschiedlicher Repräsentation vor. Untersuchen Sie die Datenmengen und ändern Sie Ihre Query so ab, dass die Seriennummern korrekt verknüpft werden. 
2. Überführen Sie diese Query (aus 2b) 1) in eine View. Sie haben Schreibrecht im Workspace `dfs.tmp`. Legen Sie darin eine View namens `labels_<ihre Ilias-ID>` an.<br>Beispiel: Lautet Ihre Ilias-ID `mr1337s` nutzen Sie `labels_mr1337s`. Falls diese View bereits existiert soll sie überschrieben werden!
3. Nutzen Sie Ihre neu angelegte View nun und zeigen Sie alle Sensoren des Besitzers `Elsen` an.
4. Wann ein Sensor eines unbekannten Besitzers (`unknown`) zum ersten Mal gesendet?
5. Lassen Sie sich den physischen Plan für die Query aus 2b 4. anzeigen. 

## 2b: 1.) Verknüpfen von Sensor Daten

- REGEXP_REPLACE(columns[1], '^s_|_$', '') entfernt das 's_' am Anfang und den '_' am Ende, aus der serial number. Dadurch werden die identifiaktionsnummern normalisiert.
- LOWER() wandelt die Seriennummern in Kleinbuchstaben um.
- DISTINCT stellt sicher, nur eindeutige Seriennummern zurückzugeben.
- Mit dem FULL OUTER JOIN werden die normalisierten Seriennummern aus den CO2-Daten mit den entsprechenden Informationen (Name, Besitzer) aus dem Datensatz bdapi.labels kombiniert.
- Mit COALESCE wird sichergestellt, dass alle Sensornamen, die existieren einen Wert haben. Wenn die serial number in der Tabelle d vorhanden ist, wird sie verwendet. Andernfalls die serial number aus l.

In [7]:
# Aufgabe 2b: 1. 
query = '''
SELECT COALESCE(d.serial_number, l.serial_number) AS serial_number,
       l.name,
       l.owner
FROM (
    SELECT DISTINCT LOWER(REGEXP_REPLACE(columns[1], '^s_|_$', '')) AS serial_number
    FROM dfs.data.`co2data.tsv`
) AS d
FULL OUTER JOIN (
    SELECT LOWER(serial) AS serial_number,
           name,
           owner
    FROM bdapi.labels
) AS l ON d.serial_number = l.serial_number
'''
csr = con.cursor()
csr.execute(query)
res = csr.fetchall()
for row in res:
    print(row)

('8caab57a6dd9_288065', None, None)
('3c6105d3abae_299589', None, None)
('0013a20041a2f0eb', 'Office H215', 'Remmy')
('0013a20041a94c97', 'Infrastructure S2', 'Unknown')
('serial_number', None, None)
('0013a20041a94ff3', 'Commerce Center S4', 'Unknown')
('0013a20041a94ca6', 'Main S1', 'Elsen')
('0013a20041a787bd', 'Main S2', 'Elsen')
('0013a20041a94ca8', 'Moving S2', 'Elsen')
('0013a20041a94be7', 'Mensa S4', 'Elsen')
('0013a20041a95193', 'Mensa S5', 'Elsen')
('3c6105d3abae', 'Main Station S5', 'Elsen')
('e8db84c5f33d_0xdeadbeef', None, None)
('d8bfc014724e_262793', None, None)
('0013a20041a94cbc', 'Moving S1', 'Elsen')
('0013a20041a94c5f', 'Main S2', 'Elsen')
('0013a20041a950cb', 'Infrastructure S1', 'Unknown')
('0013a20041a2f800', 'Office F420', 'Elsen')
('8caab57a6dd9', 'Admin S3', 'Remmy')
('8caab57cc961', 'Main Station S3', 'Elsen')
('e8db84c5f33d', 'Commerce Center S6', 'Unknown')
('0013a20041a94c04', 'Mensa S3', 'Elsen')
('d8bfc0147061', 'Commerce Center S8', 'Unknown')
('8caab57

## 2b: 2.) Erstellen einer View

- Das Resultat aus der Abfrage von 2b: 1.) wird als eine temporäre View gespeichert.
- Es wird eine View erstellt oder aktualisiert(mit CREATE OR REPLACE) mit dem Namen labels_bf1215s im Verzeichnis dfs.tmp.

In [8]:
# Aufgabe 2b: 2.
query = '''
CREATE OR REPLACE VIEW dfs.tmp.`labels_bf1215s` AS
SELECT COALESCE(d.serial_number, l.serial_number) AS serial_number,
       l.name,
       l.owner
FROM (
    SELECT DISTINCT LOWER(REGEXP_REPLACE(columns[1], '^s_|_$', '')) AS serial_number
    FROM dfs.data.`co2data.tsv`
) AS d
FULL OUTER JOIN (
    SELECT LOWER(serial) AS serial_number,
           name,
           owner
    FROM bdapi.labels
) AS l ON d.serial_number = l.serial_number
'''
csr = con.cursor()
csr.execute(query)
res = csr.fetchall()
for row in res:
    print(row)

(True, "View 'labels_bf1215s' replaced successfully in 'dfs.tmp' schema")


## 2b: 3.) Abfrage der View mit Filterung nach Owner

- Durch die Where Abfrage wird das Ergebnis auf alle Owner mit dem Namen "Elsen" reduziert.

In [9]:
# Aufgabe 2b: 3.
query = '''
SELECT * 
FROM dfs.tmp.`labels_bf1215s`
WHERE owner = 'Elsen'
'''
csr = con.cursor()
csr.execute(query)
res = csr.fetchall()
for row in res:
    print(row)

('0013a20041a94fb9', 'External S1', 'Elsen')
('0013a20041a94fa5', 'External S2', 'Elsen')
('0013a20041a94fa3', 'Office H101', 'Elsen')
('8caab57cc961', 'Main Station S3', 'Elsen')
('0013a20041a94c04', 'Mensa S3', 'Elsen')
('d8bfc014724e', 'Main Station S1', 'Elsen')
('10521c0202ab', 'Main Station S4', 'Elsen')
('0013a20041a94c62', 'Moving S3', 'Elsen')
('0013a20041a94ca6', 'Main S1', 'Elsen')
('0013a20041a787bd', 'Main S2', 'Elsen')
('0013a20041a94ca8', 'Moving S2', 'Elsen')
('0013a20041a94be7', 'Mensa S4', 'Elsen')
('0013a20041a95193', 'Mensa S5', 'Elsen')
('3c6105d3abae', 'Main Station S5', 'Elsen')
('0013a20041a94cbc', 'Moving S1', 'Elsen')
('0013a20041a94c5f', 'Main S2', 'Elsen')
('0013a20041a2f800', 'Office F420', 'Elsen')
('8caab57c3e19', 'Main Station S2', 'Elsen')
('0013a20041a78580', 'External S3', 'Elsen')
('0013a20041a94c19', 'Mensa S1', 'Elsen')
('0013a20041a951ad', 'Mensa S2', 'Elsen')


## 2b: 4.) Abfrage frühester Zeitstempel von "unknown" owner

- Join verknüpft die beiden Datensätze miteinander(da co2.data die Timestamps beinhaltet und die View die Owner-Daten).
- Die Funktion FROM_UNIXTIME wandelt den Unix-Zeitstempel in ein lesbares Datumsformat.
- Mit Where wird nach dem owner "unknown" gesucht.
- Mit Min wird der früheste Zeitstempel gefunden.

In [10]:
# Aufgabe 2b: 4. 
query = '''
SELECT MIN(FROM_UNIXTIME(CAST(c.columns[6] AS BIGINT))) as timestamp
FROM dfs.data.`co2data.tsv` AS c
JOIN dfs.tmp.`labels_bf1215s` AS l
ON CONCAT('s_', l.serial_number, '_') = c.columns[1]
WHERE LOWER(l.owner) = 'unknown'
'''
csr = con.cursor()
csr.execute(query)
res = csr.fetchall()
for row in res:
    print(row)

('2021-04-10 18:42:42', )


## 2b: 5.) Ausgabe des physischen Plans

- Durch "Explain plan for" wird der physische Plan von 2b: 4.) ausgegeben.
- Durch den physischen Plan kann beobachtet werden, was für Schritte die Datenbank unternimmt um der Query gerecht zu werden.

In [11]:
# Aufgabe 2b: 5.
query = '''
EXPLAIN PLAN FOR
SELECT MIN(FROM_UNIXTIME(CAST(c.columns[6] AS BIGINT))) as first_timestamp
FROM dfs.data.`co2data.tsv` AS c
JOIN dfs.tmp.`labels_bf1215s` AS l
ON CONCAT('s_', l.serial_number, '_') = c.columns[1]
WHERE LOWER(l.owner) = 'unknown'
'''
csr = con.cursor()
csr.execute(query)
res = csr.fetchall()
for row in res:
    print(row)

('00-00    Screen\n00-01      Project(first_timestamp=[$0])\n00-02        ComplexToJson\n00-03          Project(first_timestamp=[$0])\n00-04            StreamAgg(group=[{}], first_timestamp=[MIN($0)])\n00-05              StreamAgg(group=[{}], first_timestamp=[MIN($0)])\n00-06                Project($f0=[FROM_UNIXTIME(CAST($1):BIGINT)])\n00-07                  HashJoin(condition=[=($0, $2)], joinType=[inner], semi-join: =[false])\n00-09                    Project($f2=[ITEM($0, 1)], ITEM=[ITEM($0, 6)])\n00-10                      Scan(table=[[dfs, data, co2data.tsv]], groupscan=[EasyGroupScan [selectionRoot=file:/opt/data/bda1/co2data.tsv, numFiles=1, columns=[`columns`[1], `columns`[6]], files=[file:/opt/data/bda1/co2data.tsv], usedMetastore=false, limit=-1]])\n00-08                    BroadcastExchange\n01-01                      Project($f3=[CONCAT(\'s_\', CASE(IS NOT NULL($0), $0, $1), \'_\')])\n01-02                        SelectionVectorRemover\n01-03                          Filte

## Aufgabe 2 c

1. Die Datenmenge unter `dfs.bdapi.sensorslastday` ändert sich in Intervallen.  Finden Sie einen Weg, diese Datenmenge nutzbar zu machen. (Tipp: Drill bietet Funktionen für komplexe Datentypen an, z.B. [FLATTEN()](https://drill.apache.org/docs/flatten/)). Die Antwort Ihrer Query sollte die Felder `timewindow`, `celsius` und `humidity` liefern.
2. Geben Sie mit einer einzigen Query die beiden Datenpunkte der extremen Temperaturen (`celsius`) aus, d.h. die Zeile mit der höchsten Temeperatur und die Zeile mit der niedrigsten Temperatur.
3. Optional: Finden Sie für die Datenmenge `dfs.bdapi.sensorslastday` zu jedem Zeitraum heraus, ob es Tag ist oder Nacht. Das können Sie mithilfe der Datenmenge `dfs.weather.sunrise` tun. Nehmen Sie für den Sensor in der Datenmenge die Geokoordinaten `50.75410842895508`, `6.08587121963501` an.  

## 2c: 1.) JSON-Datenabfrage

- Das JSON-Array resultset wird mit der FLATTEN-Funktion entpackt. Dabei wird jedes Objekt im Array als separate Zeile angezeigt.
- Das Flatten der JSON-Daten ermöglicht es, die Datenstruktur in eine tabellarische Form zu bringen und die Daten einfach abzufragen.
- Die Abfrage gibt 'timewindow', 'celsius' und 'humidity' für alle Messungen zurück.

In [36]:
# Aufgabe 2c: 1. 
query = '''
SELECT t.read.timewindow AS timewindow,
t.read.humidity AS humidity,
t.read.celsius AS celsius
FROM (
    SELECT FLATTEN(resultset) AS read
    FROM bdapi.sensorslastday
) AS t
LIMIT 10
'''
csr = con.cursor()
csr.execute(query)
res = csr.fetchall()
for row in res:
    print(row)

('2023-05-26 01:45:00', 34.3306109111, 23.3846352895)
('2023-05-26 01:45:00', 37.080242157, 20.9130204519)
('2023-05-26 01:30:00', 34.1472651164, 23.421614329)
('2023-05-26 01:30:00', 36.880057017, 20.9593747457)
('2023-05-26 01:15:00', 34.1124242147, 23.5257161458)
('2023-05-26 01:15:00', 36.79773585, 21.0313154856)
('2023-05-26 01:00:00', 36.8978176117, 21.0810546875)
('2023-05-26 01:00:00', 33.8465576172, 23.5929031372)
('2023-05-26 00:45:00', 33.8612263997, 23.6533857981)
('2023-05-26 00:45:00', 37.0559196472, 21.1427726746)


## 2c: 2.) Ermittlung maximale und minimale Temperatur

- Diese Abfrage verwendet die MAX- und MIN-Funktion, um die höchste und  die niedrigste Temperatur im  Datensatz zu finden.
- Das JSON-Array resultset wird mit der FLATTEN-Funktion entpackt.
-  Die Ergebnisse werden als 'max_temperature' und 'min_temperature' ausgegeben.

In [37]:
# Aufgabe 2c: 2.
query = '''
SELECT
  MAX(t.read.celsius) AS max_temperature,
  MIN(t.read.celsius) AS min_temperature
FROM (
  SELECT FLATTEN(resultset) AS read
  FROM bdapi.sensorslastday
) AS t
'''
csr = con.cursor()
csr.execute(query)
res = csr.fetchall()
for row in res:
    print(row)

(27.5705076853, -34.4796854655)


In [ ]:
# Aufabe 2c: 3. optional

_____